<a href="https://colab.research.google.com/github/Atrimia/search-in-playlist/blob/master/search_in_playlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
＜YouTubeの再生リスト内で動画を検索するツール＞
再生リストを指定して、その再生リスト内にある動画の中から検索内容に合致する動画をピックアップする
※YouTubeのAPIキーを取得している必要がある
※非公開の再生リストにはアクセスできない
'''


import urllib.request
import urllib.parse
import json
import tempfile
import cv2
import matplotlib.pyplot as plt


#＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊

#APIキーと、再生リストID or 再生リストのURLの保存用（空のままだと実行時に入力になる）
API_KEY = ''
PLAYLIST_ID_URL = ''

#＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊＊


#インターネット上の画像を取得する関数
def web_img(url):
    img = None
    #一時ファイルを作成
    with tempfile.NamedTemporaryFile(dir='./', delete=False) as tf:
        tf.write(urllib.request.urlopen(url).read())
        tf.seek(0)
        img = cv2.imread(tf.name)
    return img

#検索にヒットした時の処理をする関数
def hit(**kwargs):
    #動画タイトル、チャンネル、動画のURLを表示
    print('Title:'+str(kwargs['snippet']['title']))
    print('Channel:'+str(kwargs['snippet']['videoOwnerChannelTitle']))
    print('URL:'+'https://www.youtube.com/watch?v='+str(kwargs['snippet']['resourceId']['videoId']))

    #動画のサムネイルを表示
    thumbnail_url = kwargs['snippet']['thumbnails']['medium']['url']
    # BGRからRGBに変換
    img = cv2.cvtColor(web_img(thumbnail_url), cv2.COLOR_BGR2RGB)
    #座標軸を消す
    plt.axis('off')
    plt.imshow(img)
    plt.show()
    print()


while_count = 1
api_ok = False
judge = ''
space = False
close = 0
word_judge = {}
nextPageToken = ''
item_count = 0
hit_count = 0


while while_count > 0:

    playlist_id_url = ''

    #APIキーを取得
    if (API_KEY == '') and (not api_ok):
        api_key = ''
        while api_key == '':
            api_key = input('APIキー：')
    if API_KEY != '':
        api_key = API_KEY

    #再生リストIDを取得
    if PLAYLIST_ID_URL == '':
        while playlist_id_url == '':
            playlist_id_url = input('再生リストID or URL：')
    else:
        playlist_id_url = PLAYLIST_ID_URL
    playlist_id = playlist_id_url.replace('https://www.youtube.com/playlist?list=','')

    #ヒットか判定する式を作成    
    if while_count == 1:
        print('---検索演算子の説明---')
        print('【AND検索：「 」(スペース) or「AND」or「+」(半角プラス)】【OR検索：「OR」or「|」(半角パイプ)】')
        print('【NOT検索：「NOT」or「-」(半角マイナス)】 で単語をつなぐ')
        print('括弧で括って検索演算子の優先順位を変えることもできる')
        print('----------------------')
        searh_bar_origin = ''
        while searh_bar_origin == '':
            searh_bar_origin = input('検索ワード：')
        search_bar = searh_bar_origin.replace('(', ' ( ')
        search_bar = search_bar.replace(')', ' ) ')
        search_bar = search_bar.replace('（', ' ( ')
        search_bar = search_bar.replace('）', ' ) ')
        search_bar = search_bar.replace('+', ' + ')
        search_bar = search_bar.replace(' -', ' - ')
        search_bar = search_bar.replace('　-', ' - ')      
        #ワードと検索演算子を１つずつ分ける
        search_words = search_bar.split()

        #検索演算子を算術演算子に変換させた文字列の式を作成
        for word in search_words:
            if (word == 'AND') or (word == '+'):
                judge += ' * '
                space = False 
            elif (word == 'OR') or (word == '|'):
                judge += ' + '
                space = False
            elif (word == 'NOT') or (word == '-'):
                judge += ' * (not '
                space = False
                close += 1
            elif word == '(':
                if space:
                  judge += ' * ('
                else:
                  judge += ' ('
                space = False
            elif word == ')':
                judge += ') '
            else:
                word_judge[word] = True
                if space:
                    judge += " * word_judge['" + word + "']"
                else:
                    judge += "word_judge['" + word + "']"
                space = True
            if close > 0:
                close += 1
                if close == 3:
                    judge += ')'
                    close = 0


    try:
        #YouTube APIのplaylistsメソッドでplaylistの情報を取得
        param = {
            'part':'snippet',
            'id':playlist_id,
            'key':api_key
        }
        target_url = 'https://www.googleapis.com/youtube/v3/playlists?'+urllib.parse.urlencode(param)
        req = urllib.request.Request(target_url)
        with urllib.request.urlopen(req) as res:
            target_body = json.load(res)
        #再生リストのタイトルを表示
        print('Playlist Title：' + str(target_body['items'][0]['snippet']['title']))
        print()

        while True:
            #YouTube APIのplaylistItemsメソッドでplaylist内の動画情報を最大50件分取得
            param = {
                'part':'snippet',
                'playlistId':playlist_id,
                'maxResults':50,
                'pageToken':nextPageToken,
                'key':api_key
            }
            target_url = 'https://www.googleapis.com/youtube/v3/playlistItems?'+urllib.parse.urlencode(param) 
            req = urllib.request.Request(target_url)
            with urllib.request.urlopen(req) as res:
                target_body = json.load(res)
                
            #再生リスト内の動画の数を記録
            item_count += len(target_body['items'])

            for item in target_body['items']:
                #動画のタイトルか概要欄に検索ワードが含まれているか判定
                for word in word_judge.keys():
                    if (word in item['snippet']['description']) or (word in item['snippet']['title']):
                        word_judge[word] = True
                    else:
                        word_judge[word] = False

                #ヒットか判定する式にワードが入っているか/いないかの情報を入れて、数式にし、１以上ならばその動画はヒット
                if eval(judge) > 0:
                    hit(**item)
                    #ヒットした動画数を記録
                    hit_count += 1

            #再生リスト内にまだ調べていない動画があるか判定
            if 'nextPageToken' in target_body:
                nextPageToken = target_body['nextPageToken']
            else:
                print(str(item_count) + '件中' + str(hit_count) + '件ヒット')
                break


    #再生リストIDにミスがあると、再生リストのタイトルの表示でIndexErrorが起きる           
    except IndexError as err:
        print('Error：' + str(err))
        print('再生リストIDが正しく入力されていない、または、再生リストが非公開になっている可能性があります')
        if PLAYLIST_ID_URL == '':
            print('もう一度入力してください')
            while_count += 1
            api_ok = True
            continue

    except urllib.error.HTTPError as err:
        print('Error：' + str(err))
        if (err.code == 400) or (err.code == 403) or (err.code == 404):
            print('APIキー、または、再生リストIDが正しく入力されていない可能性があります')
            if (API_KEY == '') or (PLAYLIST_ID_URL == ''):
                print('もう一度入力してください')
                while_count += 1
                continue

    except Exception as err:
        print('Error：' + str(err))
    
    break